# Analyzing Franz Kafka's _The Metamorphosis_ with NLP

* _Metamorphosis_ is commonly interpreted as giving insights into the meaning of personal identity, a uniquely human theme
* The conclusions that we as humans can draw from philosophical texts are limited by our preconceptions and rhetorical weaknesses
* Guiding question: Can statistical approaches from machine learning be applied to interpret _The Metamorphosis_, and how will the results support or contrast with our human reading?
* Read on to see how computers read philosophy...

In [66]:
import nltk
import numpy as np
import regex as re
import pandas as pd
from matplotlib import pyplot as plt

The text is sourced from Project Gutenberg (David Wyllie
 translation): http://www.gutenberg.org/files/5200/5200-h/5200-h.htm

In [67]:
file = open("5200-h.txt", "r")
content = file.read()

In [68]:
words = re.findall(r"[\w']+", content)

In [69]:
len(words)

22055

In [70]:
def word_freqs(words):
    freqs = {}
    for word in words:
        if word in freqs:
            freqs[word] += 1
        else:
            freqs[word] = 1
    return freqs

In [71]:
word_freqs = word_freqs(words)

In [72]:
word_freqs_to_cols = {'word':list(word_freqs.keys()), 'count':list(word_freqs.values())}
word_freqs_df = pd.DataFrame.from_dict(word_freqs_to_cols)

In [73]:
word_freqs_df.sort_values(by='count',ascending=False,inplace=True)

In [74]:
word_freqs_df.reset_index(drop=True)

,count,word
0,1097,the
1,753,to
2,612,and
3,524,his
4,495,he
5,429,of
6,406,was
7,350,had
8,339,in
9,324,that
